In [1]:
import time, os, random, re, requests
from io import BytesIO
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.by import By
from PIL import Image

from selenium.common.exceptions import NoSuchElementException,WebDriverException
import warnings
warnings.filterwarnings('ignore')

In [27]:
# IllegalCharacterError 해결(비혀용 문자 제거)
import string

def remove_nonprintable(text):
    if text is not None:
        return ''.join(filter(lambda x: x in string.printable, str(text)))
    else:
        return text

columns_to_clean = ['volume', 'manufacturer', 'nation', 'ingr_name', 'inspection', 'review_num', 'review_star']
for column in columns_to_clean:
    new_df[column] = new_df[column].apply(remove_nonprintable)

In [28]:
# new_df = pd.read_excel(r'D:\ODOC\jupyter_notebook\제품데이터_FULL\화장품 통합 데이터_유니크_V3_plus_sub.xlsx')
new_df.to_excel(r'D:\ODOC\jupyter_notebook\제품데이터_FULL\화장품 통합 데이터_유니크_V3_plus_sub.xlsx',
               index=False)

In [12]:
# merged_df = pd.merge(df, new_df[['product_code', 'image_path', 'volume', 'manufacturer', 'nation', 'ingr_name', 'inspection', 'review_num', 'review_star', 'detail_image_path']], 
#                      on='product_code', how='inner')
# new_df = merged_df

In [3]:
def product_detail_data(url):
    driver.get(url)

    first_link = driver.find_element(By.CSS_SELECTOR, "#buyInfo > a")
    first_link.click()   #클릭
    time.sleep(random.uniform(0.3, 0.5))

    volume = XPATH('//*[@id="artcInfo"]/dl[2]/dd')   #용량
    manufacturer = XPATH('//*[@id="artcInfo"]/dl[6]/dd')   #제조사
    nation = XPATH('//*[@id="artcInfo"]/dl[7]/dd')   #제조국가
    ingr = XPATH('//*[@id="artcInfo"]/dl[8]/dd')   #성분
    inspection = XPATH('//*[@id="artcInfo"]/dl[9]/dd')   #심사여부

    second_link = driver.find_element(By.CSS_SELECTOR, "#reviewInfo > a")
    second_link.click()
    time.sleep(random.uniform(0.3, 0.5))

    #리뷰 개수/점수
    review_num = XPATH('//*[@id="gdasContentsArea"]/div/div/div/div[2]/p[1]')
    review_star = XPATH('//*[@id="gdasContentsArea"]/div/div/div/div[2]/p[2]/strong')
    
    return volume, manufacturer, nation, ingr, inspection, review_num, review_star

In [4]:
def CSS(selector):
    try:
        return driver.find_element(By.CSS, selector).text
    except: return None 
    
def XPATH(path):
    try:
        return driver.find_element(By.XPATH, path).text
    except: return None 

In [1]:
#위에 함수들로 크롤링 돌리는 코드!
from selenium.common.exceptions import TimeoutException
driver = webdriver.Chrome()

idx = new_df[new_df.ingr_name.isna()].index

#칼럼별로 None값 확인해서 빈 행 채우기
for i in idx:   #범위(행) 설정: 행 인덱스로 넣음
    try:
        data = product_detail_data(new_df.LINK[i])
        
        new_df['volume'].iloc[i] = data[0]
        new_df['manufacturer'].iloc[i] = data[1]
        new_df['nation'].iloc[i] = data[2]
        new_df['ingr_name'].iloc[i] = data[3]
        new_df['inspection'].iloc[i] = data[4]
        
        if pd.isnull(new_df.review_num.iloc[i]) or new_df.review_num.iloc[i] == '':
            new_df['review_num'].iloc[i] = data[5]
            new_df['review_star'].iloc[i] = data[6]
    except (Exception,TimeoutException) as e:
        print(e)
driver.quit()

In [4]:
# new_df[1000:][new_df[:].ingr_name.isna()]   #빈 값 확인

In [47]:
#제품 사진 로컬 다운로드 -> 다운ok
base_dir = r'D:\ODOC\jupyter_notebook\제품데이터_FULL\img\plus'
for index, row in new_df.iterrows():
    try:
        image_name = f"{row['product_code']}.png"
        image_path = os.path.join(base_dir, image_name)
        image_url = row['images']

        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        new_df.at[index, 'image_path'] = 'img/'+image_name
    except Exception as e:
        print(f"Error downloading image for {image_name}: {e}")

In [53]:
#추가 데이터 _plus_sub에 저장
new_df.to_excel(r'D:\ODOC\jupyter_notebook\제품데이터_FULL\화장품 통합 데이터_유니크_V3_plus_sub.xlsx',
                index=False)

In [88]:
import ast
#엑셀 저장하면서 문자열로 바뀐 리스트 원상복구
idx = new_df['detail_images'].isna()
new_df['detail_images'][~idx] = new_df['detail_images'][~idx].apply(lambda x: ast.literal_eval(x))

In [5]:
from PIL import Image
from io import BytesIO

#제품 상세페이지 사진 다운로드 -> 상세페이지 중 누락값이 많아서 아직 다운x
base_dir = r'D:\ODOC\jupyter_notebook\제품데이터_FULL\detail_img'

#none값 제외
idx = ~(new_df.detail_images.isna())

for index, row in new_df[idx].iterrows():
    paths=[]
    for i, image_url in enumerate(row['detail_images']):
        try:
            if image_url.endswith('.jpg'):
                image_name = f"{row['product_code']}_d{i}.jpg"
            elif image_url.endswith('.gif'):
                image_name = f"{row['product_code']}_d{i}.gif"
            else:
                image_name = f"{row['product_code']}_d{i}.png"
            
            image_path = os.path.join(base_dir, image_name)
            #디렉토리에 이미 파일 있으면 스킵
            if os.path.exists(image_path): continue
                
            response = requests.get(image_url)
            if response.status_code != 200: print('FAIL: ',image_url)   #응답 성공/실패

            #gif인 경우 첫 장면 캡처해서 저장
            if image_url.endswith('.gif'):
                with Image.open(BytesIO(response.content)) as img:
                    img.seek(0)  #첫 프레임
                    img.convert('RGBA').save(image_path, 'PNG')
            else:
                with open(image_path, 'wb') as f:
                    f.write(response.content)

            paths.append('detail_img/'+image_name)
        except Exception as e:
            print(f"Error downloading image for {image_name}: {e}")
            print(image_url,'\n')
    new_df.at[index, 'detail_image_path'] = paths

In [6]:
new_df[['detail_images','detail_image_path']][:1000]